In [27]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Normalize

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

## Device

In [28]:
# Get gpu device

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.backends.cudnn.benchmark = True
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='NVIDIA GeForce GTX 1080 Ti', major=6, minor=1, total_memory=11263MB, multi_processor_count=28)

## Download CIFAR100 dataset

In [29]:
# Normalize a tensor image with mean and standard deviation of 0.5
transform = Compose([ToTensor(), Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]).to(device)])

# Download train data
training_data = datasets.CIFAR100(
    root="data",
    train=True,
    download=True,
    transform=transform,
)

# Download test data
test_data = datasets.CIFAR100(
    root="data",
    train=False,
    download=True,
    transform=transform,
)

Files already downloaded and verified
Files already downloaded and verified


### Number of classes

In [30]:
n_classes = len(test_data.classes)
n_classes

100

## Dataloaders

In [31]:
batch_size = 512
image_size = 32 * 32 * 3

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([512, 3, 32, 32])
Shape of y: torch.Size([512]) torch.int64


In [32]:
print(len(train_dataloader.dataset))
print(len(test_dataloader.dataset))

50000
10000


## Define model

In [33]:

print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self, hidden_layers=[512, 256, 128], activations=[nn.ReLU(), nn.ReLU(), nn.ReLU()], n_classes=2):
        super().__init__()
    
        self.flatten = nn.Flatten()

        layers = []

        layers.append(nn.Linear(image_size, hidden_layers[0]))
        layers.append(nn.BatchNorm1d(hidden_layers[0]))
        layers.append(activations[0])
        
        for i in range(len(hidden_layers)-1):
            layers.append(nn.Linear(hidden_layers[i], hidden_layers[i+1]))
            layers.append(nn.BatchNorm1d(hidden_layers[i+1]))
            layers.append(activations[i+1])
            
        layers.append(nn.Linear(hidden_layers[-1], n_classes))
        
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        x = self.flatten(x)
        logits = self.layers(x)
        return logits

Using cuda:0 device


## Define model

In [34]:
model = NeuralNetwork(hidden_layers = [1024, 512, 256, 128], activations=[nn.ReLU(), nn.ReLU(), nn.ReLU(), nn.ReLU()], n_classes=n_classes).to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Linear(in_features=256, out_features=128, bias=True)
    (10): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Linear(in_features=128, out_features=100, bias=True)
  )
)


## Define optimizer and loss function

In [35]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## Learning rate schedule (One cycle policy)

In [36]:
import torch.optim.lr_scheduler as lr_scheduler
 
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 0.001, epochs=10, steps_per_epoch=len(train_dataloader))

## Training loop

In [37]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        scheduler.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [38]:
def test(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
   
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n lr: {optimizer.param_groups[0]['lr']}")
    print("_______________________________")

In [39]:
epochs = 10

for i in range(epochs):
    print(f"Epoch {i+1}\n")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn, optimizer)
print("Done!")

Epoch 1



loss: 4.722047  [  512/50000]
Test Error: 
 Accuracy: 12.9%, Avg loss: 4.019726 
 lr: 0.0002814872356618626
_______________________________
Epoch 2

loss: 4.053090  [  512/50000]
Test Error: 
 Accuracy: 18.1%, Avg loss: 3.599253 
 lr: 0.0007629652551990839
_______________________________
Epoch 3

loss: 3.492297  [  512/50000]
Test Error: 
 Accuracy: 20.4%, Avg loss: 3.365861 
 lr: 0.0009999947568893096
_______________________________
Epoch 4

loss: 3.138508  [  512/50000]
Test Error: 
 Accuracy: 23.7%, Avg loss: 3.175457 
 lr: 0.0009494864126890464
_______________________________
Epoch 5

loss: 2.863136  [  512/50000]
Test Error: 
 Accuracy: 25.5%, Avg loss: 3.077332 
 lr: 0.0008099521679911583
_______________________________
Epoch 6

loss: 2.608394  [  512/50000]
Test Error: 
 Accuracy: 26.8%, Avg loss: 3.039744 
 lr: 0.0006090284912332703
_______________________________
Epoch 7

loss: 2.384428  [  512/50000]
Test Error: 
 Accuracy: 28.4%, Avg loss: 2.954768 
 lr: 0.000386510780764467